In [12]:
import pandas as pd
import numpy as np
import re

In [72]:
df=pd.read_csv('labeled_data.csv')

In [73]:
df["text"]=df["Title"]+ df["Snippet"]
#combine title and snippet into one column

In [74]:
df.drop(columns=['Title', 'Snippet'], inplace=True)
#drop all other columns

In [76]:
label_mapping = {"Medical assistance":0,
    "Shelter request":1,
    "Supplies needed":2,
    "Evacuation support":3,
    "Rescue operations":4,
    "Mental health support":5,
    "Infrastructure repair":6,
    "Animal rescue assistance":7,
    "No assistance needed":8,
    "Authority intervention":9}

df['label'] = df['Label'].map(label_mapping)

In [77]:
print(df.isnull().sum())
#checking if there are empty rows

Label    0
text     4
label    0
dtype: int64


In [78]:
df = df.dropna()
#removing empty rows

In [79]:
df[df.isnull().any(axis=1)]
#checking if we removed them

,Label,text,label


In [80]:
#turn into lower case
df['text'] = df['text'].str.lower()

In [81]:
#remove links
df['text'] = df['text'].apply(lambda x: re.sub(r"http\S+|www\S+|https\S+", "", x, flags=re.MULTILINE))

In [82]:
#remove html tags
df['text'] = df['text'].apply(lambda x: re.sub(r"<.*?>", "", x))

In [83]:
#remove non alphanumeric characters
df['text'] = df['text'].apply(lambda x: re.sub(r"[^a-zA-Z0-9\s]", "", x))

In [84]:
df.head()

,Label,text,label
0,Authority intervention,north korean troops deploy to russias kursk re...,9
1,No assistance needed,the most important issue is life itselfunfortu...,8
2,No assistance needed,celebrity chef boards the uss tripoli to judge...,8
3,Medical assistance,ksrelief provides medical assistance in lebano...,0
4,Authority intervention,germany wrestles with its arms sales to israel...,9


In [85]:
df.drop(columns='Label',inplace=True)

In [86]:
#remove extra spaces
df['text'] = df['text'].apply(lambda x: re.sub(r"\s+", " ", x).strip())  

In [87]:
df.head()

,text,label
0,north korean troops deploy to russias kursk re...,9
1,the most important issue is life itselfunfortu...,8
2,celebrity chef boards the uss tripoli to judge...,8
3,ksrelief provides medical assistance in lebano...,0
4,germany wrestles with its arms sales to israel...,9


In [66]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ENVY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [88]:
#removing english stopwords like a the and....
stop_words = set(stopwords.words("english"))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

In [89]:
df.head()

,text,label
0,north korean troops deploy russias kursk regio...,9
1,important issue life itselfunfortunately lot h...,8
2,celebrity chef boards uss tripoli judge navys ...,8
3,ksrelief provides medical assistance lebanon j...,0
4,germany wrestles arms sales israellegal ideolo...,9


In [92]:
# Save the clean labeled dataset to a new CSV file
df.to_csv('clean_labeled_news_dataset.csv', index=False)

In [90]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

c:\Users\ENVY\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#loading model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(label_mapping))

c:\Users\ENVY\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ENVY\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ber

In [93]:
#tokenizing for BERT model
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512): #512 is the max length for tokens in bert
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        
        # Tokenize the text
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long)
        }

In [94]:
#fix this pleaseee
from sklearn.model_selection import train_test_split

# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42
)

# Create the dataset instances
train_dataset = NewsDataset(train_texts, train_labels, tokenizer)
val_dataset = NewsDataset(val_texts, val_labels, tokenizer)
